In [2]:
import requests
import pandas as pd

In [3]:
file = 'C:\\Users\\mjjyo\\OneDrive\\Desktop\\supreme_court.csv'
court_data = pd.read_csv(file)
names = court_data['name']

In [4]:
names

0       HALLIBURTON OIL WELL CEMENTING CO. v. WALKER e...
1            FUNK BROTHERS SEED CO. v. KALO INOCULANT CO.
2       YOUNG v. UNITED STATES ex rel. VUITTON ET FILS...
3                UNITED STATES v. GLAXO GROUP LTD. et al.
4               UNITED STATES v. NEW WRINKLE, INC. ET AL.
                              ...                        
8798    STATE OF NEBRASKA v. STATES OF WYOMING AND COL...
8799     WASHINGTON et al. v. GENERAL MOTORS CORP. et al.
8800    SOUTH CAROLINA v. REGAN, SECRETARY OF THE TREA...
8801    SOUTH CAROLINA v. BAKER, SECRETARY OF THE TREA...
8802                 OREGON v. MITCHELL, ATTORNEY GENERAL
Name: name, Length: 8803, dtype: object

In [5]:
import govinfo 
print(dir(govinfo))

['GovInfo', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'cachetools', 'datetime', 'pytz', 'scrapelib']


In [6]:
from govinfo import GovInfo
print(dir(GovInfo))

['BASE_URL', '__annotations__', '__attrs__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_format_time', '_pages', '_throttle', '_throttled', 'accept_response', 'average_request_time', 'close', 'collections', 'congressional_hearings', 'delete', 'disable_compression', 'get', 'get_adapter', 'get_redirect_target', 'head', 'key_for_request', 'merge_environment_settings', 'mount', 'options', 'patch', 'post', 'prepare_request', 'put', 'rebuild_auth', 'rebuild_method', 'rebuild_proxies', 'request', 'requests_per_minute', 'reset_stats', 'resolve_redirects', 'retry_attempts', 'send', 'should_cache_response', 'should_strip_auth', 'urlretrieve', 

In [7]:
api_key = 'redacted'
gov = GovInfo(api_key = api_key)


In [8]:
import json
# Get a list of all available collections
collections_request = gov.collections()
collections_request_json = json.dumps(collections_request)
collections_request_dict = json.loads(collections_request_json)

for collection in collections_request_dict['collections']:
    print(collection)

{'collectionCode': 'BILLS', 'collectionName': 'Congressional Bills', 'packageCount': 249014, 'granuleCount': None}
{'collectionCode': 'BILLSTATUS', 'collectionName': 'Congressional Bill Status', 'packageCount': 138603, 'granuleCount': None}
{'collectionCode': 'BUDGET', 'collectionName': 'United States Budget', 'packageCount': 337, 'granuleCount': 6759}
{'collectionCode': 'CCAL', 'collectionName': 'Congressional Calendars', 'packageCount': 5197, 'granuleCount': 82879}
{'collectionCode': 'CDIR', 'collectionName': 'Congressional Directory', 'packageCount': 125, 'granuleCount': 15024}
{'collectionCode': 'CDOC', 'collectionName': 'Congressional Documents', 'packageCount': 16430, 'granuleCount': 9379}
{'collectionCode': 'CFR', 'collectionName': 'Code of Federal Regulations', 'packageCount': 5801, 'granuleCount': 6812498}
{'collectionCode': 'CHRG', 'collectionName': 'Congressional Hearings', 'packageCount': 35477, 'granuleCount': 286}
{'collectionCode': 'COMPS', 'collectionName': 'Statutes Co

In [68]:
# Set collection code for federal court cases
supreme_court_collection_code = 'USCOURTS'
start_date = '2022-01-01'
offset_val = 0

packages_request = gov.get(f'{gov.BASE_URL}/published/{start_date}?offset={offset_val}&pageSize=1000&collection={supreme_court_collection_code}&offsetMark=%2A&api_key={api_key}')
packages_request_dict = packages_request.json()


In [69]:
import zipfile
import urllib.request
import io
import os
import glob
import PyPDF2

In [70]:

#function for getting case file of a text, given an API link

def get_case_text(link):
    url = api_link
    response = requests.get(url)

    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
        # Get the name of the first file in the zip file
        first_file = zip_file.namelist()[1]
#         print(first_file)

        first_file_contents = zip_file.read(first_file)
        # Create a PDF reader object from the contents of the first file
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(first_file_contents))

        # Get the number of pages in the PDF file
        num_pages = len(pdf_reader.pages)
        
        all_text = []
         # Extract the text from each page in the PDF file
        for i in range(num_pages):
            # Get the page object
            page = pdf_reader.pages[i]

            # Extract the text from the page
            text = page.extract_text()
            all_text.append(text)
            
        new_all_text = ' '.join(all_text)
        return new_all_text
    
# case_text = get_case_text(api_link)
# print(case_text)

In [71]:
data = {'title':[], 'text':[]}
text_df = pd.DataFrame(data) 

In [72]:
text_df

,title,text


In [74]:
def get_court_cases(json):
    
    final_list = []
    
    sorted_packages = sorted(json['packages'], key=lambda x: x['dateIssued'])
    new_sorted_packages = [{'title': package['title'],'id': package['packageId']} for package in sorted_packages if (package['packageId'].split('-')[1] in ['ca1', 'ca2', 'ca3', 'ca4', 'ca5', 'ca6', 'ca7', 'ca8', 'ca9', 'ca10', 'ca1', 'ca13'])]
    
    
    for package in new_sorted_packages:
        api_link = f"https://api.govinfo.gov/packages/{package['id']}/zip?api_key={api_key}"

        text_df.loc[len(text_df.index)] = [package['title'], get_case_text(api_link)]
    
    
#     if json['nextPage'] is not None:
#         new_request = gov.get(json['nextPage'])
#         next_item = new_request.json()
#         get_court_cases(next_item)
#     else:
#         print('stop')
get_court_cases(packages_request_dict)

In [78]:
text_df.loc[0]['text']

"1 1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 IN \nTHE  UNITED STATES  DISTRICT  COURT  \nFOR  THE  NORTHERN MARIANA ISLANDS  \nPACIFIC  RIM  LAND  DEVELOPMENT,  \nLLC , \nPlaintiff,  \nvs. \nIMPERIAL  PACIFIC  INTERNATIONAL  \n(CNMI),  LLC , \nDefendant.  Case  No.: 19-cv-00016 \nDECISION AND ORDER DENYING  \nDEFENDANT’S  MOTION  TO DISMISS  \nAND  \nGRANTING  PLAINTIFF’S  MOTION  \nFOR  PARTIAL  SUMMARY JUDGMENT  \nI. I\nNTRODUCTION\nThis civil action involves a dispute between a general contractor, Plaintiff Pacific Rim Land \nDevelopment, LLC (“Pacific Rim”), and the  company who hired it to work on a casino construction \nproject, Defendant Imperial Pacific International (CNMI), LLC (“IPI”) . Pacific Rim initiated this \nlawsuit with a complaint against IPI for breach of contract, both of the construction contract between \nthe two parties and a promissory note executed by IPI in favor of Pacific Rim  after they agr